In [1]:
from models import backbone

_b = 'vgg'
b = backbone(_b)

In [2]:
model = b.pretrained()
print(model.summary())

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [3]:
model.trainable

True

In [4]:
model.name

'vgg16'

In [5]:
from tensorflow.keras import Model

In [6]:
model2 = Model(model.input, model.layers[-2].output)
print(model2.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [8]:
model2.layers[-1].output_shape

(None, 14, 14, 512)

In [155]:
import math
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, concatenate, Dense

In [177]:
def spatial_pyramid_pool(x, num_sample, previous_conv_size, out_pool_size):
#     print(type(num_sample))
#     num_sample=64
    for i in range(len(out_pool_size)):
        h, w = previous_conv_size
        h_window = math.ceil(h / out_pool_size[i])
        w_window = math.ceil(w / out_pool_size[i])
        h_stride = math.floor(h / out_pool_size[i])
        w_stride = math.floor(w / out_pool_size[i])
        
#         print(h_window, w_window, h_stride, w_stride)
#         print('hello')
        max_pool = MaxPooling2D((h_window, w_window), strides=(h_stride, w_stride))(x)
#         x = max_pool(previous_conv)
#         h_strd = h_size = math.ceil(float(previous_conv_size[0]) / out_pool_size[i])
#         w_strd = w_size = math.ceil(float(previous_conv_size[1]) / out_pool_size[i])
#         pad_h = int(out_pool_size[i] * h_size - previous_conv_size[0])
#         pad_w = int(out_pool_size[i] * w_size - previous_conv_size[1])
#         new_previous_conv = tf.pad(previous_conv, tf.constant([[0, 0], [0, pad_h], [0, pad_w], [0, 0]]))
#         max_pool = tf.nn.max_pool(new_previous_conv,
#                    ksize=[1,h_size, h_size, 1],
#                    strides=[1,h_strd, w_strd,1],
#                    padding='SAME')
#         print(max_pool)
        if (i == 0):
#             mrcnn_bbox = layers.Reshape((-1, num_classes, 4), name="mrcnn_bbox")(x)
            spp = Flatten()(max_pool)
        else:
            spp = concatenate([spp, Flatten()(max_pool)])
        print(spp)
  
    return spp


In [178]:
def make_model(img_input):
    x = Conv2D(96, (7, 7), strides=(2, 2), padding='same', activation='relu')(img_input)
    x = BatchNormalization()(x)
    x = MaxPooling2D(3, strides=(2, 2))(x)
    x = Conv2D(256, (5, 5), strides=(2, 2), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(3, strides=(2, 2))(x)
    x = Conv2D(384, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = Conv2D(384, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    
    spp_inp_shape = x.shape
    out_pool_size = [4, 2, 1]
    spp = spatial_pyramid_pool(x, spp_inp_shape[0], [spp_inp_shape[1], spp_inp_shape[2]], out_pool_size)
    
    return spp

In [179]:
tf.keras.backend.clear_session()

img_input = Input(shape=(224, 224, 3))

model3 = make_model(img_input)

x = Dense(4096, activation='relu')(model3)
x = Dense(4096, activation='relu')(x)
output = Dense(102, activation='relu')(x)

model = Model(img_input, output)

Tensor("flatten/Identity:0", shape=(None, 4096), dtype=float32)
Tensor("concatenate/Identity:0", shape=(None, 5120), dtype=float32)
Tensor("concatenate_1/Identity:0", shape=(None, 5376), dtype=float32)


In [176]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 96) 14208       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 112, 112, 96) 384         conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 55, 55, 96)   0           batch_normalization[0][0]        
______________________________________________________________________________________________